# Pydantic fundamentals

### Without Pydantic

In [48]:
class Person:
    def __init__(self, name: str,gender: str ,age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age

person1 = Person(name="Hampus", gender="M", age=24)
person1


In [49]:
person1.name, person1.age, person1.gender

('Hampus', 24, 'M')

In [50]:
person2 = Person(name=3621+2819, gender=True, age="Twenty four")
person2


## Validation of our person class

In [51]:
class Person:
    def __init__(self, name: str,gender: str ,age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"Name must be a string not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age
   
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self,age):
        if not isinstance(age,(int,float)):
            raise TypeError(f"Age must be an integer not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"Age must be between 0 and 124 not {age}")  
        self._age = age
    
    def __repr__(self):
        return f"Person({self.name}, {self.age}, {self.gender})"
    

p3 = Person(name="Wilma", gender="F", age= 24)
p3

Person(Wilma, 24, F)

In [52]:
p3.age, p3.name

(24, 'Wilma')

In [53]:
p3.age = -5

ValueError: Age must be between 0 and 124 not -5

# Validate with pydantic

In [ ]:
from pydantic import BaseModel, ValidationError


#Students är en vanlig klass men är också en Pydantic base modell som vi inheritar från
class Student(BaseModel):
    name: str
    gender: str
    age: int

s1 = Student(name="Eva", gender="F", age=7)
s1

Student(name='Eva', gender='F', age=7)

In [ ]:
s1.age = 55
s1.name = "Maria"
s1

Student(name='Maria', gender='F', age=55)

In [ ]:
try:
    Student(name=2021, gender="F", age=7)
except ValidationError as e:
    print(e)

1 validation error for Student
name
  Input should be a valid string [type=string_type, input_value=2021, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [ ]:
from pydantic import Field
from typing import Literal


#Students är en vanlig klass men är också en Pydantic base modell som vi inheritar från
class Student(BaseModel):
    name: str
    gender: Literal["M","F"] # gender måste vara antingen "M" eller "F"
    age: int = Field(gt = -1, lt=125)  # age måste vara mellan 0 och 124

try:
    Student(name="Johan", gender="x", age =-1)
except ValidationError as e:
    print(e)

2 validation errors for Student
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='x', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


In [ ]:
s5 = Student(name="Johan", gender="M", age =1)
s5

Student(name='Johan', gender='M', age=1)

In [ ]:
s5.age = -10
s5

Student(name='Johan', gender='M', age=-10)

## Serialization and deserialization
- Serialization convert object -> JSON
- Deserialization convert JSON -> Object

In [ ]:
s5.model_dump()

{'name': 'Johan', 'gender': 'M', 'age': -10}

In [ ]:
import pandas as pd
pd.read_json('housing.json')

,address,price,area,rooms
0,"Sveavägen 98, Norrmalm",5200000,34,1.5
1,"Hornsgatan 72, Södermalm",7600000,52,2.0
2,"Värtavägen 12, Östermalm",11200000,78,3.0
3,"Götgatan 112, Södermalm",8900000,65,2.5
4,"Hälsingegatan 4, Vasastan",6700000,44,1.5


In [ ]:
import json
with open('housing.json') as f:
    data = json.load(f)

data

[{'address': 'Sveavägen 98, Norrmalm',
  'price': 5200000,
  'area': 34,
  'rooms': 1.5},
 {'address': 'Hornsgatan 72, Södermalm',
  'price': 7600000,
  'area': 52,
  'rooms': 2},
 {'address': 'Värtavägen 12, Östermalm',
  'price': 11200000,
  'area': 78,
  'rooms': 3},
 {'address': 'Götgatan 112, Södermalm',
  'price': 8900000,
  'area': 65,
  'rooms': 2.5},
 {'address': 'Hälsingegatan 4, Vasastan',
  'price': 6700000,
  'area': 44,
  'rooms': 1.5}]

In [55]:
from pydantic import BaseModel, Field
class House(BaseModel):
    address: str
    price: int = Field(lt = 10_000_000)
    area: float
    rooms: float

In [56]:
# House(address='....', price=5200.., area=34, rooms=1.5)
House(**data[0])

House(address='Sveavägen 98, Norrmalm', price=5200000, area=34.0, rooms=1.5)

In [58]:
[House(**house_data) for house_data in data]

ValidationError: 1 validation error for House
price
  Input should be less than 10000000 [type=less_than, input_value=11200000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than